## 数据处理

In [7]:
###引入函数###

import pandas as pd
import io
import re
from datetime import datetime, timedelta
from typing import List, Union, Optional
from datetime import datetime

import os, re, json, time
import typing as T
import requests
from pathlib import Path
from openpyxl import Workbook, load_workbook

from openpyxl.utils import get_column_letter
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from data_processing import load_and_process,build_jsonl_for_range, save_jsonl
from model_classify import postprocess_excel_by_topic

In [8]:
## 研发字典
speaker_map = {
    "16186514":   "研发peter本尊",
    "1655611808": "运营绾绾",
    "2073820674": "沙利文老师",
}
## 客服字典
MAPPING_FILE = "mapping地球1.xlsx"

##QQ的txt文件
pathtxt      = "sample2_100.txt"

# 1) 拿到 JSONL（列表）
jsonl_lines01 = build_jsonl_for_range(
    pathtxt=pathtxt,
    mapping_file=MAPPING_FILE,
    speaker_map=speaker_map,
    start_time=None,  # 不筛
    end_time=None,    # 不筛
    return_str=False,
)

print(len(jsonl_lines01), "lines")
print(jsonl_lines01[0])

115 lines
{"发言日期": "2025-08-06", "发言时间": "16:35:34", "玩家ID": "枫桥夜柏(854411209)", "玩家消息": "彩虹源质是其他几个源质的平均值"}


## 大模型分析

In [9]:
"""
批处理 10000 条聊天数据（每批 100 条）：
- 模型#1：过滤非游戏相关（只保留相关 JSON 行，原样输出）
- 模型#2：对保留内容进行 5 类意图分类（输出带“意图分类”的 JSON 行）
- 结果按 sheet（体感反馈/疑惑不解和问题询问/玩家建议和灵感/情绪输出/问题反馈）写入 Excel
"""
from model_classify import (
    load_system_prompt,
    build_user_prompt_filter, build_user_prompt_classify,build_user_prompt_classify2,
    call_ark_chat_completions,
    jsonl_to_dataframe_with_intent,
    create_intent_excel_styled,append_json_to_excel_by_cat_and_tag,
)

## 设置参数

In [10]:
# ============= 你的模型与文件配置（改这里） =============
API_URL   = "https://ark.cn-beijing.volces.com/api/v3/chat/completions" 
API_KEY = "de91deb0-aae6-46cb-bac0-17ac3b6107f5" #API
V3_MODEL_ID= "ep-20251020160142-5d7hp"#接入点
V3_1_MODEL_ID = "ep-20251020160025-9p5tj"#接入点
R1_MODEL_ID = "ep-20251020160103-5n6g2"#接入点

PROMPT_MD_PATH01 = Path("提示词1.md")      # 模型#1 system 提示词（筛相关）
PROMPT_MD_PATH02 = Path("提示词2.md")      # 模型#2 system 提示词（做分类）
PROMPT_MD_PATH03 = Path("提示词3.md")      # 模型#3 system 提示词（做二级标签）
PROMPT_MD_PATH05 = Path ("提示词5_直接话提簇分类.md")
EXCEL_FILE       = Path("testmerge.xlsx")   # 输出 Excel
BATCH_SIZE       = 100
SLEEP_BETWEEN    = 1   # 每批之间的间隔，防止QPS触发限流；按需调整
RETRIES          = 2
TEMPERATURE      = 0.20
MAX_TOKENS       = 16384
TIMEOUT_SEC      = 600
# =====================================================

## 开始运行

In [11]:
from tqdm import tqdm
import time

# ... 前置：系统提示、create_intent_excel_styled(EXCEL_FILE)、jsonl_lines01 等

system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)  # 筛相关
system_prompt02 = load_system_prompt(PROMPT_MD_PATH02)  # 做分类
system_prompt03 = load_system_prompt(PROMPT_MD_PATH03)  # 做分类
system_prompt05 = load_system_prompt(PROMPT_MD_PATH05)
create_intent_excel_styled(EXCEL_FILE)

total = len(jsonl_lines01)
if total == 0:
    print("没有可处理的数据。")
else:
    total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"准备处理 {total} 条，共 {total_batches} 批（每批 {BATCH_SIZE} 条）。")

written_total = 0

for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    batch_lines = jsonl_lines01[start:end]

    try:
        # --- 模型 #1：筛相关 ---
        user_prompt1 = build_user_prompt_filter(batch_lines)
        output_filter = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_MODEL_ID,
            system_prompt=system_prompt01,
            user_prompt=user_prompt1,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        if not output_filter:
            tqdm.write(f"[批次 {b+1}] 模型#1 无有效输出，跳过。")
            continue
        # --- 模型 #2：分类 ---
        user_prompt2 = build_user_prompt_classify(output_filter)
        output_classify = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt02,
            user_prompt=user_prompt2,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        if not output_classify:
            tqdm.write(f"[批次 {b+1}] 模型#2 无有效输出，跳过。")
            continue
        # --- 模型 #3：二级标签 ---
        user_prompt3 = build_user_prompt_classify2(output_classify)
        output_classify2 = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt05,
            user_prompt=user_prompt3,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )

        # 转 DataFrame
        df_batch = jsonl_to_dataframe_with_intent(output_classify2)
        if df_batch is None or df_batch.empty:
            tqdm.write(f"[批次 {b+1}] ⚠️ 无可写入数据（空 DataFrame）。")
            continue

        # ✅ 关键：转为 list[dict] 再写入
        records = df_batch.to_dict(orient="records")
        append_json_to_excel_by_cat_and_tag(records, EXCEL_FILE)

        batch_written = len(records)
        written_total += batch_written
        tqdm.write(f"[批次 {b+1}] ✅ 写入 {batch_written} 条。")
        print(output_classify2)
  
    except Exception as e:
        tqdm.write(f"[批次 {b+1}] ❌ 出错：{e}")
        continue

    time.sleep(SLEEP_BETWEEN)
postprocess_excel_by_topic(EXCEL_FILE, gap_minutes=30, nat_policy="skip")

print("\n✅ 全部批次处理完成！")
print(f"输入总数：{total}")
print(f"写入总数：{written_total}")


✅ 创建并套样式：testmerge.xlsx
准备处理 115 条，共 2 批（每批 100 条）。


🔥 批处理进度:   0%|                                                                                                                                                                                                          | 0/2 [07:37<?, ?批/s]      

[批次 1] ✅ 写入 93 条。
{"发言日期":"2025-08-06","发言时间":"16:35:34","玩家ID":"枫桥夜柏(854411209)","玩家消息":"彩虹源质是其他几个源质的平均值","分类标签":1,"话题簇":"源质机制"}
{"发言日期":"2025-08-06","发言时间":"16:35:37","玩家ID":"ゞ兲迯萊客ゞ(37307559)","玩家消息":"都不流畅","分类标签":1,"话题簇":"性能问题"}
{"发言日期":"2025-08-06","发言时间":"16:35:43","玩家ID":"枫桥夜柏(854411209)","玩家消息":"其他源质上去了，彩虹自动就上去了","分类标签":1,"话题簇":"源质机制"}
{"发言日期":"2025-08-06","发言时间":"16:35:58","玩家ID":"东风(2790800575)","玩家消息":"不是，彩虹是最低的那个","分类标签":1,"话题簇":"源质机制"}
{"发言日期":"2025-08-06","发言时间":"16:36:04","玩家ID":"东风(2790800575)","玩家消息":"不是平均","分类标签":1,"话题簇":"源质机制"}
{"发言日期":"2025-08-06","发言时间":"16:36:11","玩家ID":"小罗别睡了(814562830)","玩家消息":"@枫桥夜柏 不是平均，是最低的那个","分类标签":1,"话题簇":"源质机制"}
{"发言日期":"2025-08-06","发言时间":"16:36:12","玩家ID":"什么(3549173353)","玩家消息":"[图片]","分类标签":1,"话题簇":"图片类发言"}
{"发言日期":"2025-08-06","发言时间":"16:36:39","玩家ID":"炎殇魔尊燯弑天(1140885081)","玩家消息":"第一个精神崩溃","分类标签":1,"话题簇":"精神状态"}
{"发言日期":"2025-08-06","发言时间":"16:36:40","玩家ID":"枫桥夜柏(854411209)","玩家消息":"我上一测就是养的希伯特鲎","分类标签":1,"话题簇":"殖装选择"}
{"发言日期":"2025-0

🔥 批处理进度:  50%|████████████████████████████████████████████████████████████████████████████████████████████████▌                                                                                                | 1/2 [08:55<07:38, 458.30s/批]      

[批次 2] ✅ 写入 14 条。
{"发言日期": "2025-08-06", "发言时间": "16:50:24", "玩家ID": "虞罗(2816328364)", "玩家消息": "这个是固定的", "分类标签": 1, "话题簇": "机制特性"}
{"发言日期": "2025-08-06", "发言时间": "16:50:34", "玩家ID": "虞罗(2816328364)", "玩家消息": "有可能可以升级", "分类标签": 1, "话题簇": "机制特性"}
{"发言日期": "2025-08-06", "发言时间": "16:51:02", "玩家ID": "ツ笙歌绝丶何以笙箫默(2723600969)", "玩家消息": "我升到四级，但是好像没什么变化", "分类标签": 1, "话题簇": "机制特性"}
{"发言日期": "2025-08-06", "发言时间": "16:51:15", "玩家ID": "吾辈是猫君莫笑(2317313105)", "玩家消息": "[图片]", "分类标签": 1, "话题簇": "机制特性"}
{"发言日期": "2025-08-06", "发言时间": "16:51:31", "玩家ID": "虞罗(2816328364)", "玩家消息": "开荒好用", "分类标签": 1, "话题簇": "机制特性"}
{"发言日期": "2025-08-06", "发言时间": "16:51:36", "玩家ID": "虞罗(2816328364)", "玩家消息": "后面就不行了", "分类标签": 1, "话题簇": "机制特性"}
{"发言日期": "2025-08-06", "发言时间": "16:51:43", "玩家ID": "东风(2790800575)", "玩家消息": "[图片]", "分类标签": 1, "话题簇": "机制特性"}
{"发言日期": "2025-08-06", "发言时间": "16:52:00", "玩家ID": "✨(2806047279)", "玩家消息": "玩不懂", "分类标签": 2, "话题簇": "玩法理解"}
{"发言日期": "2025-08-06", "发言时间": "16:52:02", "玩家ID": "虞罗(2816328364

🔥 批处理进度: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [08:56<00:00, 268.31s/批]

✅ 已完成后处理：testmerge.xlsx（gap=30min, NaT策略=skip）

✅ 全部批次处理完成！
输入总数：115
写入总数：107



E:\项目\玩家社群分析智能体\测试样本-合并完全\model_classify.py:520: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ts = pd.to_datetime(col, errors="coerce")  # 宽松解析
E:\项目\玩家社群分析智能体\测试样本-合并完全\model_classify.py:520: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ts = pd.to_datetime(col, errors="coerce")  # 宽松解析
E:\项目\玩家社群分析智能体\测试样本-合并完全\model_classify.py:520: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ts = pd.to_datetime(col, errors="coerce")  # 宽松解析
